In [44]:
!pip install   requests icecream suds-py3



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [42]:
from zeep import Client, Settings
from zeep.cache import SqliteCache
from zeep.transports import Transport
from requests import Session
import xml.etree.ElementTree as ET

def create_soap_client(wsdl_url):
    # Configure settings to handle missing schemas
    settings = Settings(
        strict=False,
        xml_huge_tree=True
    )

    # Setup caching and transport
    session = Session()
    cache = SqliteCache(path='.soap_cache.sqlite')
    transport = Transport(cache=cache, session=session)

    # Create client with custom settings
    return Client(
        wsdl_url,
        transport=transport,
        settings=settings
    )

def find_tijuana_station(client):
    # Get station codes XML
    station_codes_xml = client.service.exportStationCodesXML()

    # Parse XML response
    root = ET.fromstring(station_codes_xml)

    # Search for Tijuana station
    for station in root.findall('.//Station'):
        if 'Tijuana' in station.findtext('StationName', ''):
            return station.findtext('StationCode', '')

    return None

def get_station_params(client, station_code):
    # Get all parameters for the station
    return client.service.exportAllParamsXML(station_code)


https://cdmo.baruch.sc.edu/dges/

wsdl: https://cdmo.baruch.sc.edu/webservices2/requests.cfc?wsdl

In [13]:


wsdl = "https://cdmo.baruch.sc.edu/webservices2/requests.cfc?wsdl"



In [43]:
wsdl_url = "https://cdmo.baruch.sc.edu/webservices2/requests.cfc?wsdl"

try:
    # Create SOAP client
    client = create_soap_client(wsdl_url)

    # Find Tijuana station
    station_code = find_tijuana_station(client)
    if station_code:
        print(f"Found Tijuana station code: {station_code}")

        # Get parameters for the station
        params = get_station_params(client, station_code)
        print("Station parameters:")
        print(params)
    else:
        print("Tijuana station not found")

except Exception as e:
    print(f"Error: {e}")


Error: Service has no operation 'exportStationCodesXML'


/Users/valentin/development/dev_earthcube/infrastructuredashboard/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportSingleParamXMLResponse' contains an invalid part ('exportSingleParamXMLReturn'): invalid xsd type or elements
  warnings.warn(str(exc))
/Users/valentin/development/dev_earthcube/infrastructuredashboard/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportAllParamsDateRangeXMLNewResponse' contains an invalid part ('exportAllParamsDateRangeXMLNewReturn'): invalid xsd type or elements
  warnings.warn(str(exc))
/Users/valentin/development/dev_earthcube/infrastructuredashboard/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportAllParamsDateRangeXMLResponse' contains an invalid part ('exportAllParamsDateRangeXMLReturn'): invalid xsd type or elements
  warnings.warn(str(exc))


It seems pysimplesoap package installation failed. Let's try installing it with a different package name - the correct package name is 'PySimpleSOAP' (note the capitalization).

After installing the package correctly, continue with the existing code that uses pysimplesoap. If you still experience issues, we can try using the zeep client that is already working.


In [48]:
from suds.client import Client
from suds.cache import DocumentCache
import xml.etree.ElementTree as ET

def get_station_data():
    # Create SUDS client with caching
    wsdl_url = "https://cdmo.baruch.sc.edu/webservices2/requests.cfc?wsdl"
    cache = DocumentCache()
    client = Client(wsdl_url, cache=cache)

    try:
        # Print available methods to verify correct names
        print("Available methods:")
        print(client)

        # Get list of stations (use actual method name from service)
        result = client.service.exportStationCodesXML()

        # Parse XML response
        root = ET.fromstring(result)

        # Search for Tijuana station
        for station in root.findall('.//Station'):
            station_name = station.findtext('StationName', '')
            if 'Tijuana' in station_name:
                station_code = station.findtext('StationCode', '')
                print(f"Found station: {station_name} (Code: {station_code})")

                # Get parameters using the correct method name
                params = client.service.getStationParameters(station_code)
                print("\nStation parameters:")
                print(params)
                return

        print("Tijuana station not found")

    except Exception as e:
        print(f"Error: {e}")

# Run the function
get_station_data()

Available methods:

Suds ( https://github.com/cackharot/suds-py3 )  version: 1.4.5.0 IN  build: 20211115

Service ( RequestsService ) tns="http://webservices2"
   Prefixes (2)
      ns0 = "http://rpc.xml.coldfusion"
      ns1 = "http://schemas.xmlsoap.org/soap/encoding/"
   Ports (1):
      (requests.cfc)
         Methods (11):
            NERRFilterStationCodesXML(xs:string NERRFilter, )
            NERRFilterStationCodesXMLNew(xs:string NERRFilter, )
            exportAllParamsDateRangeXML(xs:string Station_Code, xs:string minDate, xs:string maxDate, xs:string param, )
            exportAllParamsDateRangeXMLNew(xs:string Station_Code, xs:string minDate, xs:string maxDate, xs:string param, xs:string wildcard, )
            exportAllParamsXML(xs:string Station_Code, xs:string recs, )
            exportAllParamsXMLNew(xs:string Station_Code, xs:string recs, xs:string wildcard, )
            exportSingleParamXML(xs:string Station_Code, xs:string recs, xs:string param, )
            expor

In [42]:
from zeep import Client
from zeep.cache import SqliteCache
from zeep.transports import Transport
from requests import Session

# Create a session with caching
session = Session()
cache = SqliteCache(path='.cache.sqlite')
transport = Transport(cache=cache, session=session)

# Initialize SOAP client
client = Client(wsdl, transport=transport)
client.wsdl.dump()
# Example: Print available operations
#print("Available operations:", list(client.service._operations.keys()))



Prefixes:
     xsd: http://www.w3.org/2001/XMLSchema
     ns0: http://rpc.xml.coldfusion

Global elements:
     

Global types:
     xsd:anyType
     ns0:CFCInvocationException()
     xsd:ENTITIES
     xsd:ENTITY
     xsd:ID
     xsd:IDREF
     xsd:IDREFS
     xsd:NCName
     xsd:NMTOKEN
     xsd:NMTOKENS
     xsd:NOTATION
     xsd:Name
     xsd:QName
     xsd:anySimpleType
     xsd:anyURI
     xsd:base64Binary
     xsd:boolean
     xsd:byte
     xsd:date
     xsd:dateTime
     xsd:decimal
     xsd:double
     xsd:duration
     xsd:float
     xsd:gDay
     xsd:gMonth
     xsd:gMonthDay
     xsd:gYear
     xsd:gYearMonth
     xsd:hexBinary
     xsd:int
     xsd:integer
     xsd:language
     xsd:long
     xsd:negativeInteger
     xsd:nonNegativeInteger
     xsd:nonPositiveInteger
     xsd:normalizedString
     xsd:positiveInteger
     xsd:short
     xsd:string
     xsd:time
     xsd:token
     xsd:unsignedByte
     xsd:unsignedInt
     xsd:unsignedLong
     xsd:unsignedShort

Bindings:

/Users/valentin/development/dev_resilient/resilient_workflows/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportSingleParamXMLResponse' contains an invalid part ('exportSingleParamXMLReturn'): invalid xsd type or elements
  warnings.warn(str(exc))
/Users/valentin/development/dev_resilient/resilient_workflows/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportAllParamsDateRangeXMLNewResponse' contains an invalid part ('exportAllParamsDateRangeXMLNewReturn'): invalid xsd type or elements
  warnings.warn(str(exc))
/Users/valentin/development/dev_resilient/resilient_workflows/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportAllParamsDateRangeXMLResponse' contains an invalid part ('exportAllParamsDateRangeXMLReturn'): invalid xsd type or elements
  warnings.warn(str(exc))
/Users/valen

In [9]:
!pip install suds-py3


  Obtaining dependency information for suds-py3 from https://files.pythonhosted.org/packages/8a/39/aa44074c42b3ee2d2ce36c2cd891787f9ae4158416600390dd09d09233da/suds_py3-1.4.5.0-py3-none-any.whl.metadata
  Using cached suds_py3-1.4.5.0-py3-none-any.whl.metadata (778 bytes)
Using cached suds_py3-1.4.5.0-py3-none-any.whl (298 kB)

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


I'll help create a SUDS client for the SOAP web service. Since `suds-py3` package is already installed in the last cell, let's create a client using it.



In [33]:
from suds.client import Client as SudsClient
from suds.cache import DocumentCache

# Initialize SUDS client with caching
cache = DocumentCache()
client = SudsClient(url=wsdl, cache=cache)

# Print available methods
print(client)


Suds ( https://github.com/cackharot/suds-py3 )  version: 1.4.5.0 IN  build: 20211115

Service ( RequestsService ) tns="http://webservices2"
   Prefixes (2)
      ns0 = "http://rpc.xml.coldfusion"
      ns1 = "http://schemas.xmlsoap.org/soap/encoding/"
   Ports (1):
      (requests.cfc)
         Methods (11):
            NERRFilterStationCodesXML(xs:string NERRFilter, )
            NERRFilterStationCodesXMLNew(xs:string NERRFilter, )
            exportAllParamsDateRangeXML(xs:string Station_Code, xs:string minDate, xs:string maxDate, xs:string param, )
            exportAllParamsDateRangeXMLNew(xs:string Station_Code, xs:string minDate, xs:string maxDate, xs:string param, xs:string wildcard, )
            exportAllParamsXML(xs:string Station_Code, xs:string recs, )
            exportAllParamsXMLNew(xs:string Station_Code, xs:string recs, xs:string wildcard, )
            exportSingleParamXML(xs:string Station_Code, xs:string recs, xs:string param, )
            exportSingleParamXMLNew(

In [35]:
from icecream import ic
def get_service_info():
    # Get service information
    print("Services:")
    print("-" * 50)
    for service in client.wsdl.services:
        try:
            print(f"Service: {service.name}")
            for port in service.ports:
                print(f"\nPort: {port.name}")
                print("\nMethods:")
                for op in port.methods.values():
                    print(f"- {op}")
        except Exception as e:
            print(f"Error: {e}")


get_service_info()

Services:
--------------------------------------------------
Service: RequestsService

Port: requests.cfc

Methods:
- (Method){
   name = "exportAllParamsXML"
   location = b'https://cdmo.baruch.sc.edu/webservices2/requests.cfc'
   binding = 
      (binding){
         input = <suds.bindings.rpc.Encoded object at 0x11276ecc0>
         output = <suds.bindings.rpc.Encoded object at 0x11276ecc0>
      }
   soap = 
      (soap){
         action = """"
         style = "rpc"
         input = 
            (Input){
               body = 
                  (Body){
                     parts[] = 
                        (Part){
                           root = <wsdl:part name="Station_Code" type="xsd:string">

      </wsdl:part>
                           name = "Station_Code"
                           qname[] = 
                              "Station_Code",
                              "http://webservices2",
                           element = "None"
                           type = "('str

The code above:
1. Creates a SUDS client with document caching
2. Displays available services and methods
3. Makes a sample SOAP request using one of the available methods

Note: The actual method names and parameters will depend on the WSDL specification. The `exportStationCodesXML` is just an example - you should replace it with the specific method you need to call.


To handle the missing `http://xml.apache.org/xml-soap` schema and configure a proxy for SOAP requests, we'll set up the SUDS client with proxy settings and custom namespace handling.

The code above:
1. Creates an Import object for the missing schema
2. Sets up schema doctor to handle the missing namespace
3. Configures proxy settings (optional)
4. Creates a new SUDS client with the schema handling


In [37]:
client.service.exportStationCodesXML()

TypeNotFound: Type not found: '(Document, http://xml.apache.org/xml-soap, )'